In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.io.kafka import ReadFromKafka
from apache_beam.io.gcp.datastore.v1new.datastoreio import WriteToDatastore

# Define las opciones del pipeline
options = PipelineOptions(streaming=True)

# Configura el topic y los datos de Kafka
topic = 'test_topic'
bootstrap_servers = 'kafka:9092'
options.view_as(beam.options.pipeline_options.StandardOptions).streaming = True

# Define el pipeline
with beam.Pipeline(options=options) as p:
    # Lee los mensajes desde Kafka
    messages = (p
                | "ReadFromKafka" >> ReadFromKafka(
                    consumer_config={
                        'bootstrap.servers': bootstrap_servers,
                        'auto.offset.reset': 'earliest'
                    },
                    topics=[topic],
                )
               )

    # Define las transformaciones
    result = (messages
              | "TransformData" >> beam.Map(lambda message: (message.key, message.value))
              | "WriteToDatastore" >> WriteToDatastore(project_id="my-project", kind="my-kind"))

    # Ejecuta el pipeline
    result | "PrintResult" >> beam.Map(print)
